In [89]:
import pandas as pd
import numpy as np
import pickle as pkl
import os,sys

module_path = os.path.abspath(os.path.join('../CATENets/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import catenets.models.torch.pseudo_outcome_nets as pseudo_outcome_nets

/homes/gws/mingyulu/miniconda3/envs/torch_gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [85]:
# shuffle = True

results = pkl.load(open("../results/sprint/model_selection_shuffle_True.pkl", "rb"))

In [86]:
for learner, learner_results in results.items():
    print(learner)
    
    for sec in ["if_pehe", "pseudo_outcome_r", "pseudo_outcome_dr"]:
        
        print(learner, sec, np.mean(learner_results[sec]), np.std(learner_results[sec]))
    

XLearner
XLearner if_pehe -7.3220899479682515 34.360372304324876
XLearner pseudo_outcome_r 0.2634781399338269 0.011139601581523356
XLearner pseudo_outcome_dr 0.513934977042968 0.023511302666828632
DRLearner
DRLearner if_pehe 29.67226623077074 39.091255350510934
DRLearner pseudo_outcome_r 0.26831529017633643 0.011595430158978242
DRLearner pseudo_outcome_dr 0.5244584839026472 0.024418306858262285
SLearner
SLearner if_pehe -42.61655667012481 32.45610126811392
SLearner pseudo_outcome_r 0.25930062103608326 0.012011401540573443
SLearner pseudo_outcome_dr 0.5052254059353722 0.02521167590587563
TLearner
TLearner if_pehe -21.003707193449806 23.69559224625513
TLearner pseudo_outcome_r 0.2603571344652835 0.011854579936811273
TLearner pseudo_outcome_dr 0.5076786762534126 0.02483914441698314
RLearner
RLearner if_pehe -36.36550233005499 32.49211077481245
RLearner pseudo_outcome_r 0.26003925500210373 0.012012430783341615
RLearner pseudo_outcome_dr 0.5071391428750776 0.025252498758941894
RALearner
RAL